In [16]:
from __future__ import absolute_import
from __future__ import print_function

import os
import argparse
import pandas as pd
import random
import csv
random.seed(49297)
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
output_path = '../../data/in-hospital-mortality'
root_path = '../../data/root'

In [216]:
def get_detailed_stats(partition, n_hours = 24):
    output_dir = os.path.join(output_path, str(n_hours), partition)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    all_stays_df = pd.read_csv(os.path.join(root_path, 'all_stays.csv'))
    adms_df = pd.read_csv('../../data/ADMISSIONS.csv')

#     ethinicity = all_stays_df['ETHNICITY'].value_counts()
#     gender = all_stays_df['GENDER'].value_counts()
#     ax = ethinicity.plot.bar(x='ethinic groups', y='counts', rot=0, figsize=(25,25))
#     plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
#     ax = gender.plot.bar(x='gender groups', y='counts', rot=0, figsize=(5,5))
#     plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
    
    patients_all = list(filter(str.isdigit, os.listdir(output_dir)))
    gender_groups_all, gender_groups_positive = {},{}
    race_groups_positive, race_groups_all = {},{}
    admission_type_all,admission_type_positive = {},{}
    with open(os.path.join(output_dir,'listfile.csv')) as csvfile:
        f = csv.reader(csvfile)
        for row in f:
            patient = row[0].split('.')[0].split('_')[0]
            if patient == 'stay':
                continue
            
            label = row[1]
            
            #gender = all_stays_df.loc[all_stays_df.ICUSTAY_ID == int(icu_stay), 'GENDER'].iloc[0]
            gender = all_stays_df[all_stays_df['SUBJECT_ID'] == int(patient)]['GENDER'].values[0]
            #print(gender)
            race = all_stays_df[all_stays_df['SUBJECT_ID'] == int(patient)]['ETHNICITY'].values[0]
            admission_type = adms_df[adms_df['SUBJECT_ID'] == int(patient)]['ADMISSION_TYPE'].values[0]
            
            if race not in race_groups_all:
                race_groups_all[race] = 1
            else:
                race_groups_all[race]+=1
                
            if gender not in gender_groups_all:
                gender_groups_all[gender] = 1
            else:
                gender_groups_all[gender]+=1
            if admission_type not in admission_type_all:
                admission_type_all[admission_type] =1
            else:
                admission_type_all[admission_type]+=1
            

            if label == '1':
                if gender not in gender_groups_positive:
                    gender_groups_positive[gender] = 1
                else:
                    gender_groups_positive[gender]+=1
                if race not in race_groups_positive:
                    race_groups_positive[race] =1
                else:
                    race_groups_positive[race]+=1
                if admission_type not in admission_type_positive:
                    admission_type_positive[admission_type]=1
                else:
                    admission_type_positive[admission_type]+=1
                
                    
                    
        race_positive_percent = {}
        gender_positive_percent = {}
        adm_type_percent = {}
        
        for key in race_groups_positive:
            race_positive_percent[key] = race_groups_positive[key]/race_groups_all[key]
        race_positive_percent= {k: v for k, v in sorted(race_positive_percent.items(), key=lambda item: item[1])}
        
        for key in gender_groups_positive:
            gender_positive_percent[key] = gender_groups_positive[key]/gender_groups_all[key]
        gender_positive_percent= {k: v for k, v in sorted(gender_positive_percent.items(), key=lambda item: item[1])}
        
        for key in admission_type_positive:
            adm_type_percent[key] = admission_type_positive[key]/admission_type_all[key]
        adm_type_percent= {k: v for k, v in sorted(adm_type_percent.items(), key=lambda item: item[1])}
        
    print('positive patient is', gender_positive_percent)
    print('race positive percent', race_positive_percent)
    print('admission type', adm_type_percent)
    

In [218]:
get_detailed_stats('test')

positive patient is {'M': 0.08798929407828705, 'F': 0.10423026602703882}
race positive percent {'HISPANIC OR LATINO': 0.04225352112676056, 'PATIENT DECLINED TO ANSWER': 0.045454545454545456, 'OTHER': 0.07291666666666667, 'BLACK/AFRICAN AMERICAN': 0.07708779443254818, 'ASIAN - ASIAN INDIAN': 0.09090909090909091, 'WHITE': 0.09307875894988067, 'ASIAN': 0.10294117647058823, 'HISPANIC/LATINO - PUERTO RICAN': 0.1111111111111111, 'UNKNOWN/NOT SPECIFIED': 0.13747228381374724, 'HISPANIC/LATINO - DOMINICAN': 0.14285714285714285, 'WHITE - OTHER EUROPEAN': 0.14285714285714285, 'WHITE - RUSSIAN': 0.15, 'MULTI RACE ETHNICITY': 0.15384615384615385, 'ASIAN - CHINESE': 0.16666666666666666, 'UNABLE TO OBTAIN': 0.17391304347826086, 'BLACK/HAITIAN': 0.2, 'ASIAN - OTHER': 0.3333333333333333, 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE': 0.5, 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 0.5}
admission type {'ELECTIVE': 0.03397508493771234, 'URGENT': 0.10416666666666667, 'EMERGENCY': 0.10740305